In [1]:
import pandas as pd
import numpy as np
import os
import scipy.stats as stats

# train test split from sklearn
from sklearn.model_selection import train_test_split

#importing my acquire script
import acquire
import prepare

# Viz imports
import matplotlib.pyplot as plt

In [2]:
# reading in telco churn data using function in acquire module
telco_df = acquire.get_telco_data()


In [3]:
# # describes the df, object and numeric cols
# acquire.get_data_summary(telco_df)

# # shows distribution of individual variables
# acquire.get_viz(telco_df)

### Acquire Takeaways:
- Churn is our target variable
- payment_type_id == payment type. We will keep the payment type id column since that is numeric
- We will encode payment type to show the different payment types as numeric
- internet_service type id column is the same as internet service type column. We will keep the internet service type id column since that is numeric
- contract type id column is the same as contract type column. We will keep the contract type id column since that is numeric  and encode the contract type column to check for each contract category
- binary categorical columns like gender, partner, dependents will have to be encoded to be numeric
- total charges column has an object dtype - will have to change to float
- we will drop the customer id column 
- Some customers had no total charges -  this was where the tenure was 0 so must be new customers

# Prepare

In [4]:
telco_df = prepare.prep_telco_data(telco_df)
telco_df.head()

There are no duplicate rows in your dataset.
Removing the following columns: ['customer_id']


,payment_type_id,internet_service_type_id,contract_type_id,senior_citizen,tenure,monthly_charges,total_charges,gender_Male,has_partner,has_dependents,...,streaming_movies_Yes,paperless_billing_Yes,churn_Yes,has_1yr_contract,has_2yr_contract,has_fiber,has_internet_service,pay_by_card,pay_echeck,pay_mailcheck
0,2,1,3,0,65,90.45,5957.90,0,1,1,...,1,1,0,0,1,0,0,0,0,1
1,4,1,3,0,54,45.20,2460.55,1,0,0,...,0,0,0,0,1,0,0,1,0,0
2,3,1,3,0,56,45.05,2560.10,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,4,1,3,0,20,39.40,825.40,1,1,1,...,0,1,0,0,1,0,0,1,0,0
4,3,1,3,0,72,85.15,6316.20,1,1,0,...,1,1,0,0,1,0,0,0,0,0


In [13]:
telco_df.churn_Yes.shape

(7043,)

In [5]:
train, validate, test = prepare.train_validate_test_split(telco_df, 'churn_Yes')

### Prepare Takeaways:

- There are no duplicates in the dataset
- I dropped the customer id column 
- I changed the total charges which were empty to null values and then replaced them with the same value as the monthly charges for new customers.
- Then I changed total charges to a float
- Encoded categorical variables
- Split the data set into train, validate and test

NameError: name 'train' is not defined